First load the necessary libraries:

In [1]:
import miditapyr as mt
import mido

Then load the [midi file included in this repository](https://github.com/urswilke/miditapyr/raw/master/notebooks/test_midi_file.mid):

In [2]:
mid_file_str = 'test_midi_file.mid'
mido_mid_file = mido.MidiFile(mid_file_str)

Now the midi data can be loaded into 2 dataframes df_meta & df_notes, and an integer ticks_per_beat:

In [3]:
df_meta, df_notes, ticks_per_beat = mt.mido_midi_df(mido_mid_file)

In [4]:
df_meta

,type,name,time,tempo,numerator,denominator,clocks_per_click,notated_32nd_notes_per_beat,i_track
0,track_name,Drum Machine,0,NaN,NaN,NaN,NaN,NaN,1
3,set_tempo,nan,0,545454.0,NaN,NaN,NaN,NaN,1
4,time_signature,nan,0,NaN,4.0,4.0,24.0,8.0,1
2151,end_of_track,nan,241,NaN,NaN,NaN,NaN,NaN,1
2152,track_name,FM-4,0,NaN,NaN,NaN,NaN,NaN,2
2435,end_of_track,nan,1954,NaN,NaN,NaN,NaN,NaN,2
2436,track_name,FM-4,0,NaN,NaN,NaN,NaN,NaN,3
4969,end_of_track,nan,202,NaN,NaN,NaN,NaN,NaN,3


In [5]:
df_notes

,type,name,time,note,velocity,channel,i_track
1,note_on,nan,0,38.0,101.0,9.0,1
2,note_on,nan,0,36.0,101.0,9.0,1
5,note_off,nan,240,38.0,101.0,9.0,1
6,note_off,nan,0,36.0,101.0,9.0,1
7,note_on,nan,1200,38.0,101.0,9.0,1
...,...,...,...,...,...,...,...
4964,note_on,nan,240,54.0,102.0,0.0,3
4965,note_off,nan,0,49.0,64.0,0.0,3
4966,note_off,nan,240,54.0,64.0,0.0,3
4967,note_on,nan,0,42.0,102.0,0.0,3


In [6]:
ticks_per_beat

960

In order to use the R package [pyramidi](https://github.com/urswilke/pyramidi) (via the cell magic **%%R**), the python package rpy2 needs to be used:

In [7]:
%load_ext rpy2.ipython 

Now we are ready to run R code:

In [8]:
%%R -i df_meta -i df_notes -i ticks_per_beat
suppressMessages(library(tidyverse))
df_notes_wide <- 
    pyramidi::tab_measures(df_meta, df_notes, ticks_per_beat) %>%
    pyramidi::widen_events()
df_notes_wide

# A tibble: 2,481 x 17
   i_track name  channel  note i_note m_note_on m_note_off b_note_on b_note_off
     <int> <chr>   <dbl> <dbl>  <int>     <dbl>      <dbl>     <dbl>      <dbl>
 1       1 nan         9    38      1       0         0.25        0          1 
 2       1 nan         9    36      1       0         0.25        0          1 
 3       1 nan         9    38      2       1.5       1.75        6          7.
 4       1 nan         9    38      3       2         2.25        8          9 
 5       1 nan         9    36      2       2         2.25        8          9 
 6       1 nan         9    38      4       3.5       3.75       14.        15 
 7       1 nan         9    38      5       4         4.25       16         17.
 8       1 nan         9    36      3       4         4.25       16         17.
 9       1 nan         9    38      6       5.5       5.75       22         23.
10       1 nan         9    38      7       6         6.25       24         25 
# … with 2,471 mo

The following code imports the dataframe `df_notes_wide` back to python:

In [9]:
import pandas
from rpy2.robjects import pandas2ri, r

pandas2ri.activate()
df_notes_wide = r['df_notes_wide']
df_notes_wide



,i_track,name,channel,note,i_note,m_note_on,m_note_off,b_note_on,b_note_off,t_note_on,t_note_off,ticks_note_on,ticks_note_off,time_note_on,time_note_off,velocity_note_on,velocity_note_off
1,1,nan,9.0,38.0,1,0.00,0.250000,0.0,1.0000,0.000000,0.136363,0,240,0,240,101.0,101.0
2,1,nan,9.0,36.0,1,0.00,0.250000,0.0,1.0000,0.000000,0.136363,0,240,0,0,101.0,101.0
3,1,nan,9.0,38.0,2,1.50,1.750000,6.0,7.0000,0.818181,0.954545,1440,1680,1200,240,101.0,101.0
4,1,nan,9.0,38.0,3,2.00,2.250000,8.0,9.0000,1.090908,1.227272,1920,2160,240,240,101.0,101.0
5,1,nan,9.0,36.0,2,2.00,2.250000,8.0,9.0000,1.090908,1.227272,1920,2160,0,0,101.0,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,3,nan,0.0,49.0,141,382.75,383.000000,1531.0,1532.0000,208.772519,208.908882,367440,367680,0,240,102.0,64.0
2478,3,nan,0.0,42.0,93,383.00,383.250000,1532.0,1533.0000,208.908882,209.045246,367680,367920,0,0,102.0,64.0
2479,3,nan,0.0,49.0,142,383.25,383.500000,1533.0,1534.0000,209.045246,209.181609,367920,368160,240,0,102.0,64.0
2480,3,nan,0.0,54.0,76,383.50,383.750000,1534.0,1535.0000,209.181609,209.317973,368160,368400,240,240,102.0,64.0


Now we can visualize a piano roll of the midi data in `df_notes_wide`:

In [10]:
import altair as alt

alt.Chart(df_notes_wide).mark_bar().encode(
    x='m_note_on:T',
    x2='m_note_off:T',
    y='note:N',
    color='velocity_note_on:Q',
    tooltip=['m_note_on', 'm_note_off', 'note']
).properties(
    width=200,
    height=200
).facet(
    facet='i_track:O',
    columns=1
).resolve_scale(
    y='independent'
).interactive()

alt.FacetChart(...)

Uncomment this command to save the midi data back to a file:

In [12]:
#mt.df_2_midi(df_meta, df_notes, ticks_per_beat, "test.mid")